In [ ]:
import numpy as np

import seaborn as sn

import torch
from torchvision.datasets import EMNIST

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils

import matplotlib.pyplot as plt
%matplotlib inline

from emnist import list_datasets
from emnist import extract_training_samples
from emnist import extract_test_samples

print(list_datasets())

X_train, y_train = extract_training_samples('letters')

print("train shape:", X_train.shape)
print("train labels:", y_train.shape)

X_test, y_test = extract_test_samples('letters')

print("test shape:", X_test.shape)
print("test labels:", y_test.shape)
np.array(np.unique(y_train, return_counts=True)).T

np.array(np.unique(y_test, return_counts=True)).T

import string
def plot_sample(index):
    plt.matshow(X_train[index], cmap='binary')
    n = y_train[index] + ord('a') - 1;
    print("letter:", chr(n), "\nindex:", y_train[index])

plot_sample(0)

plot_sample(1)

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
print(X_train.shape)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
print(X_test.shape)

X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255

Y_train = np_utils.to_categorical(y_train, 27)
Y_test = np_utils.to_categorical(y_test, 27)

EPOCHS = 5
INPUT_SHAPE = (28, 28, 1)
KERNEL_SIZE = (3,3)
BATCH_SIZE = 64
DROPRATE = 0.25
CLASSES = 27
VALIDATION_DATA = (X_test, Y_test)
VERBOSE = 1
OPTIMIZER = Adam()

cnn = models.Sequential([
    layers.Convolution2D(filters=32, 
                         kernel_size=KERNEL_SIZE, 
                         padding = 'same', 
                         activation ='relu', 
                         input_shape=INPUT_SHAPE),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding = 'same'),

    layers.Convolution2D(filters=64, 
                         kernel_size=KERNEL_SIZE,
                         padding = 'same',
                         activation ='relu',
                         input_shape=INPUT_SHAPE),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding = 'same'),

    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(DROPRATE),
    layers.Dense(CLASSES, activation = 'softmax')
])

cnn.summary()

cnn.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

tb_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/base", histogram_freq=1)

history = cnn.fit(X_train, 
                  Y_train, 
                  batch_size=BATCH_SIZE, 
                  epochs=EPOCHS, 
                  verbose=VERBOSE, 
                  validation_data=VALIDATION_DATA, 
                  callbacks=[tb_callback])

%load_ext tensorboard
%tensorboard --logdir="C:/Users/pereg/-- machine learning/logs"

score = cnn.evaluate(X_test, Y_test, verbose=VERBOSE)
print("\nTest score:", score[0])
print("Test accuracy:", score[1])
print("Test accuracy: %.4f%%" % (score[1]*100))

print(history.history.keys())

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

y_predicted = cnn.predict(X_test)

y_predicted_labels = [np.argmax(i) for i in y_predicted]

cm = tf.math.confusion_matrix(labels=y_test,predictions=y_predicted_labels)

plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
